# 简单的 RAG 实现

基于 [Alfredo Deza 的 GitHub 仓库](https://github.com/alfredodeza/learn-retrieval-augmented-generation)。

在本笔记本中，我们将基于一个结构化的葡萄酒评分 CSV 文件构建一个简单的 RAG 应用程序。我们将：

* [加载数据集](#loading-the-dataset)。
* [使用向量嵌入对某一列进行编码](#encode-using-vector-embedding)。
* [**R**：Retrieve 根据用户查询，使用语义相似性检索部分行](#retrieve-sematically-relevant-data-based-on-users-query)。
* [**A**：Augment  将检索到的数据增强到 LLM 的提示中](#augment-the-prompt-to-the-llm-with-retrieved-data)。
* [**G**：Generate 根据检索到的行生成对用户查询的回复](#generate-reply-to-the-users-query)。

### 视觉改进

我们将使用 [rich 库](https://github.com/Textualize/rich) 和 `rich-theme-manager` 来使输出更易读，并抑制警告信息。

In [2]:
from rich.console import Console
from rich.style import Style
import pathlib
from rich_theme_manager import Theme, ThemeManager

THEMES = [
    Theme(
        name="dark",
        description="Dark mode theme",
        tags=["dark"],
        styles={
            "repr.own": Style(color="#e87d3e", bold=True),      # Class names
            "repr.tag_name": "dim cyan",                        # Adjust tag names 
            "repr.call": "bright_yellow",                       # Function calls and other symbols
            "repr.str": "bright_green",                         # String representation
            "repr.number": "bright_red",                        # Numbers
            "repr.none": "dim white",                           # None
            "repr.attrib_name": Style(color="#e87d3e", bold=True),    # Attribute names
            "repr.attrib_value": "bright_blue",                 # Attribute values
            "default": "bright_white on black"                  # Default text and background
        },
    ),
    Theme(
        name="light",
        description="Light mode theme",
        styles={
            "repr.own": Style(color="#22863a", bold=True),          # Class names
            "repr.tag_name": Style(color="#00bfff", bold=True),     # Adjust tag names 
            "repr.call": Style(color="#ffff00", bold=True),         # Function calls and other symbols
            "repr.str": Style(color="#008080", bold=True),          # String representation
            "repr.number": Style(color="#ff6347", bold=True),       # Numbers
            "repr.none": Style(color="#808080", bold=True),         # None
            "repr.attrib_name": Style(color="#ffff00", bold=True),  # Attribute names
            "repr.attrib_value": Style(color="#008080", bold=True), # Attribute values
            "default": Style(color="#000000", bgcolor="#ffffff"),   # Default text and background
        },
    ),
]

theme_dir = pathlib.Path("themes").expanduser()
theme_dir.expanduser().mkdir(parents=True, exist_ok=True)

theme_manager = ThemeManager(theme_dir=theme_dir, themes=THEMES)
theme_manager.list_themes()

dark = theme_manager.get("dark")
theme_manager.preview_theme(dark)

 Theme  Description       Tags  Path               
 dark   Dark mode theme   dark  themes\dark.theme  
 light  Light mode theme        themes\light.theme

                                      Theme: dark - themes\dark.theme                                      
┌───────────────────┬───────────────┬───────┬─────────┬─────────┬────────────────┬────────────────────────┐
│ style             │ color         │ color │ bgcolor │ bgcolor │ attributes     │ example                │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ default           │ bright_white  │ █████ │ black   │ █████   │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_name  │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_value │ bright_blue   │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.call         │ bright_yellow │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.none         │ white         │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.number       │ bright_red    │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.own          │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.str          │ bright_green  │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.tag_name     │ cyan          │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
└───────────────────┴───────────────┴───────┴─────────┴─────────┴────────────────┴────────────────────────┘
┌─ attributes legend ──────────────────────────────────────────────────────────────────┐
│  b: bold, d: dim, i: italic, u: underline, U: double underline, B: blink, 2: blink2  │
│  r: reverse, c: conceal, s: strike, f: frame, e: encircle, o: overline, L: Link      │
└──────────────────────────────────────────────────────────────────────────────────────┘

In [3]:
from rich.console import Console

dark = theme_manager.get("dark")
# Create a console with the dark theme
console = Console(theme=dark)


In [4]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

## 加载数据集 <a id='loading-the-dataset'></a>

由于数据是一个简单、小型且结构化的 CSV 文件，我们可以使用 Pandas 来加载它。

In [6]:
# 导入 pandas 库，用于数据处理和分析
import pandas as pd

# 读取 CSV 文件 'data/top_rated_wines.csv' 并将其加载到一个 DataFrame 中
# 使用 query 方法过滤掉 'variety' 列中值为 NaN 的行
# 使用 reset_index 方法重置索引，并丢弃旧的索引列
# 使用 to_dict 方法将 DataFrame 转换为字典列表，其中每个字典代表一行数据
data = (
    pd
    .read_csv('data/top_rated_wines.csv')  # 读取 CSV 文件
    .query('variety.notna()')  # 过滤掉 'variety' 列中值为 NaN 的行
    .reset_index(drop=True)  # 重置索引，并丢弃旧的索引列
    .to_dict('records')  # 将 DataFrame 转换为字典列表
)

# 使用 console.print 方法打印前两行数据
console.print(data[:2])

[
    {
        'name': '3 Rings Reserve Shiraz 2004',
        'region': 'Barossa Valley, Barossa, South Australia, Australia',
        'variety': 'Red Wine',
        'rating': 96.0,
        'notes': 'Vintage Comments : Classic Barossa vintage conditions. An average wet Spring followed by extreme 
heat in early February. Occasional rainfall events kept the vines in good balance up to harvest in late March 2004.
Very good quality coupled with good average yields. More than 30 months in wood followed by six months tank 
maturation of the blend prior to bottling, July 2007. '
    },
    {
        'name': 'Abreu Vineyards Cappella 2007',
        'region': 'Napa Valley, California',
        'variety': 'Red Wine',
        'rating': 96.0,
        'notes': 'Cappella is a proprietary blend of two clones of Cabernet Sauvignon with Cabernet Franc, Petit 
Verdot and Merlot. The gravelly soil at Cappella produces fruit that is very elegant in structure. The resulting 
wine exhibits beautiful purity of fruit with fine grained and lengthy tannins. '
    }
]

## 使用向量嵌入编码  <a id='encode-using-vector-embedding'></a>

我们将使用流行的开源向量数据库 [Qdrant](https://qdrant.tech/)，以及流行的嵌入编码器和文本转换库 [SentenceTransformer](https://sbert.net/)。

In [7]:
from qdrant_client import models, QdrantClient
# qdrant_client 是一个用于与 Qdrant 向量数据库进行交互的 Python 客户端库。
# models 模块包含了 Qdrant 的数据模型，QdrantClient 是用于与 Qdrant 服务器进行通信的客户端类。
from sentence_transformers import SentenceTransformer
# sentence_transformers 是一个用于生成句子嵌入（sentence embeddings）的库。
# SentenceTransformer 类用于加载预训练的模型来将文本转换为向量表示。
# 创建向量数据库客户端
qdrant = QdrantClient(":memory:")
# 创建了一个 Qdrant 客户端实例，并将其配置为在内存中运行。
# 这意味着所有的数据操作（如插入、查询等）都将在内存中进行，而不会持久化到磁盘。
# 这对于测试和开发环境非常有用，因为它可以快速启动和销毁，而不需要管理磁盘上的数据文件。

# Create the embedding encoder
encoder = SentenceTransformer('all-MiniLM-L6-v2')
# SentenceTransformer('all-MiniLM-L6-v2') 创建了一个句子嵌入编码器实例，并加载了预训练的模型 all-MiniLM-L6-v2。
# 这个模型是一个轻量级的语言模型，专门用于将句子转换为高维向量（嵌入）。
# 这些嵌入可以用于各种自然语言处理任务，如文本相似度计算、聚类、分类等

# 总结
# 这段代码的主要目的是初始化一个在内存中运行的 Qdrant 向量数据库客户端，并加载一个预训练的句子嵌入模型。
# 后续的代码可能会使用 encoder 将文本转换为向量，
# 并使用 qdrant 客户端将这些向量存储到 Qdrant 数据库中，或者进行相似度搜索等操作。


In [ ]:
# 这段代码是用于在Qdrant向量数据库中创建一个新的集合（collection），用于存储葡萄酒评分数据。
# Qdrant是一个用于向量相似性搜索的开源向量数据库。
collection_name="top_wines"

# qdrant.recreate_collection() 是Qdrant提供的一个方法，用于重新创建一个集合。
# 如果集合已经存在，它会被删除并重新创建；如果集合不存在，则会直接创建一个新的集合。
# 
# vectors_config=models.VectorParams(...)：这个参数用于配置集合中向量的属性。
# models.VectorParams 是一个类，用于定义向量的参数。
# 
# encoder.get_sentence_embedding_dimension() 是一个方法调用，返回编码器（encoder）生成的句子嵌入的维度。
# 这意味着向量的维度是由所使用的模型决定的。
# 
# distance=models.Distance.COSINE：distance 参数指定了向量之间的距离度量方式。
# 
# models.Distance.COSINE 表示使用余弦相似度作为距离度量方式。
# 余弦相似度是一种常用的向量相似度度量方法，特别适用于文本嵌入向量的比较。
qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)



### 将数据加载到向量数据库中

我们将使用上面创建的（向量）集合，遍历葡萄酒数据集的 `notes` 列，将其编码为嵌入向量，并存储到向量数据库中。在加载数据的同时，后台会运行数据索引以支持快速检索。

此步骤将花费几秒钟时间（在我的笔记本电脑上不到一分钟）。

In [ ]:
# vectorize!
qdrant.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [ ]:
console.print(qdrant.get_collection(collection_name=collection_name))

## **R**etrieve：基于用户查询检索语义相关数据 <a id='retrieve-sematically-relevant-data-based-on-users-query'></a>

一旦数据加载到向量数据库并且索引过程完成，我们就可以开始使用我们的简单 RAG 系统了。

In [ ]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

### 编码用户的查询

我们将使用与编码文档数据相同的编码器来编码用户的查询。  
通过这种方式，我们可以基于语义相似性来搜索结果。

In [ ]:
query_vector = encoder.encode(user_prompt).tolist()

### 搜索相似的行

现在，我们可以使用用户查询的嵌入编码，在向量数据库中查找相似的行。

In [ ]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3
)

In [ ]:
from rich.console import Console
from rich.text import Text
from rich.table import Table

table = Table(title="Retrieval Results", show_lines=True)

table.add_column("Name", style="#e0e0e0")
table.add_column("Region", style="bright_red")
table.add_column("Variety", style="green")
table.add_column("Rating", style="yellow")
table.add_column("Notes", style="#89ddff")
table.add_column("Score", style="#a6accd")

for hit in hits:
    table.add_row(
        hit.payload["name"],
        hit.payload["region"],
        hit.payload["variety"],
        str(hit.payload["rating"]),
        f'{hit.payload["notes"][:50]}...',
        f"{hit.score:.4f}"
    )

console.print(table)

## **A**ugment：将检索到的数据增强到 LLM 的提示中 <a id='augment-the-prompt-to-the-llm-with-retrieved-data'></a>

在我们的简单示例中，我们将直接使用前 3 个结果，并将它们原样添加到生成 LLM 的提示中。

## **G**enerate：生成对用户查询的回复 <a id='generate-reply-to-the-users-query'></a>

我们将使用 [OpenAI](https://platform.openai.com/docs/models) 中最受欢迎的生成式 AI 大语言模型之一。

In [ ]:
from dotenv import load_dotenv

load_dotenv()

### 首先，尝试不使用 **R**etrieval（检索）

我们可以仅基于用户提示让 LLM 进行推荐。

In [ ]:
# Now time to connect to the large language model
from openai import OpenAI
from rich.panel import Panel

# client = OpenAI()
client = OpenAI(api_key="sk-83db2355e64e4639ace2fbaaf75e1f4a", base_url="https://api.deepseek.com")
completion = client.chat.completions.create(
    # model="gpt-3.5-turbo",
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Here is my wine recommendation:"}
    ]
)

response_text = Text(completion.choices[0].message.content)
styled_panel = Panel(
    response_text,
    title="Wine Recommendation without Retrieval",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)

### 现在，添加 **R**etrieval（检索）结果

推荐听起来很棒，但我们的库存和菜单中没有这款葡萄酒。此外，可能有一些新葡萄酒是 LLM 预训练数据中未包含的。

我们将使用 **R**etrieval 结果运行相同的查询，以获得更符合我们业务需求的推荐。

In [ ]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [ ]:
completion = client.chat.completions.create(
    # model="gpt-3.5-turbo",
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results)}
    ]
)

response_text = Text(completion.choices[0].message.content)
styled_panel = Panel(
    response_text,
    title="Wine Recommendation with Retrieval",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)